# Created by Sunny You on September 1, 2023

This script adds in the future of each storm's time of landfall to the model's training data.

In [1]:
import numpy as np
import pandas as pd
import os
import re

In [2]:
os.chdir(os.path.dirname(os.path.dirname(os.getcwd())))
print(os.getcwd())

/Users/sunnyyou/Documents/Real_Time_HAI


In [3]:
data = pd.read_csv('BRTK_SHIPS_2000to2019_IMERG_OK_Request_2023_FINAL.csv')
data['ID'] = data.GIS_ID.str[:10]
data = data[["GIS_ID", "JULDAY","LAND_OCEAN","ID"]]
data

,GIS_ID,JULDAY,LAND_OCEAN,ID
0,ATL_200001_TD_2000060718,2451703.25,Ocean,ATL_200001
1,ATL_200001_TD_2000060800,2451703.50,Ocean,ATL_200001
2,ATL_200001_TD_2000060806,2451703.75,Ocean,ATL_200001
3,ATL_200001_TD_2000060812,2451704.00,Ocean,ATL_200001
4,ATL_200002_TD_2000062300,2451718.50,Ocean,ATL_200002
...,...,...,...,...
40973,SPA_201927_TD_2019051506,2458618.75,Land,SPA_201927
40974,SPA_201927_TD_2019051512,2458619.00,Ocean,SPA_201927
40975,SPA_201927_TD_2019051518,2458619.25,Ocean,SPA_201927
40976,SPA_201927_TD_2019051600,2458619.50,Ocean,SPA_201927


In [4]:
s = data[data.ID == "ATL_200001"]
s

,GIS_ID,JULDAY,LAND_OCEAN,ID
0,ATL_200001_TD_2000060718,2451703.25,Ocean,ATL_200001
1,ATL_200001_TD_2000060800,2451703.50,Ocean,ATL_200001
2,ATL_200001_TD_2000060806,2451703.75,Ocean,ATL_200001
3,ATL_200001_TD_2000060812,2451704.00,Ocean,ATL_200001


In [5]:
final_df = pd.DataFrame(columns = ["GIS_ID", "LOP06", "LOP12", "LOP18", "LOP24"])

i=0
for i in range(data.shape[0]):
    if i%1000 == 0:
        print(i)
    row = data.loc[i]
    cur_time = row.JULDAY
    temp = data[data.ID == row.ID]
    p6_time = cur_time + 0.25
    p12_time = cur_time + 0.5
    p18_time = cur_time + 0.75
    p24_time = cur_time + 1.0

    p6 = temp[temp.JULDAY == p6_time]
    p12 = temp[temp.JULDAY == p12_time]
    p18 = temp[temp.JULDAY == p18_time]
    p24 = temp[temp.JULDAY == p24_time]

    if p6.shape[0] == 0:
        p6 = "-999"
    else: 
        p6 = p6.LAND_OCEAN.iloc[0]

    if p12.shape[0] == 0:
        p12 = "-999"
    else: 
        p12 = p12.LAND_OCEAN.iloc[0]

    if p18.shape[0] == 0:
        p18 = "-999"
    else: 
        p18 = p18.LAND_OCEAN.iloc[0]

    if p24.shape[0] == 0:
        p24 = "-999"
    else: 
        p24 = p24.LAND_OCEAN.iloc[0]

    final_df.loc[i] = [row.GIS_ID, p6, p12, p18, p24]

final_df


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000


,GIS_ID,LOP06,LOP12,LOP18,LOP24
0,ATL_200001_TD_2000060718,Ocean,Ocean,Ocean,-999
1,ATL_200001_TD_2000060800,Ocean,Ocean,-999,-999
2,ATL_200001_TD_2000060806,Ocean,-999,-999,-999
3,ATL_200001_TD_2000060812,-999,-999,-999,-999
4,ATL_200002_TD_2000062300,Ocean,Ocean,Ocean,Ocean
...,...,...,...,...,...
40973,SPA_201927_TD_2019051506,Ocean,Ocean,Ocean,-999
40974,SPA_201927_TD_2019051512,Ocean,Ocean,-999,-999
40975,SPA_201927_TD_2019051518,Ocean,-999,-999,Ocean
40976,SPA_201927_TD_2019051600,-999,-999,Ocean,-999


In [6]:
f = final_df
f["LOP12"] = np.where(f.LOP06 != "Ocean", f.LOP06, f.LOP12)
f["LOP18"] = np.where(f.LOP12 != "Ocean", f.LOP12, f.LOP18)
f["LOP24"] = np.where(f.LOP18 != "Ocean", f.LOP18, f.LOP24)

In [7]:
f

,GIS_ID,LOP06,LOP12,LOP18,LOP24
0,ATL_200001_TD_2000060718,Ocean,Ocean,Ocean,-999
1,ATL_200001_TD_2000060800,Ocean,Ocean,-999,-999
2,ATL_200001_TD_2000060806,Ocean,-999,-999,-999
3,ATL_200001_TD_2000060812,-999,-999,-999,-999
4,ATL_200002_TD_2000062300,Ocean,Ocean,Ocean,Ocean
...,...,...,...,...,...
40973,SPA_201927_TD_2019051506,Ocean,Ocean,Ocean,-999
40974,SPA_201927_TD_2019051512,Ocean,Ocean,-999,-999
40975,SPA_201927_TD_2019051518,Ocean,-999,-999,-999
40976,SPA_201927_TD_2019051600,-999,-999,-999,-999


In [9]:
original_data = pd.read_csv('BRTK_SHIPS_2000to2019_IMERG_OK_Request_2023_FINAL.csv')
original_data

,GIS_ID,BASIN,DATE,JULDAY,LATITUDE,LONGITUDE,MSLP,STORM_NAME,STORM_ID,CAT,...,TADV_12,TADV_13,HMEAN100,RA300,HRA100,HRA300,RAIN_AXIS_100,RAIN_AXIS_300,RAIN_AXIS_100_300,RMR
0,ATL_200001_TD_2000060718,ATL,2000060718,2451703.25,21.000000,-93.000000,1008,NOT_NAMED,200001,TD,...,-999.0,-999.0,15.642342,0.504786,0.151420,0.055654,47.571724,34.547775,28.093451,85
1,ATL_200001_TD_2000060800,ATL,2000060800,2451703.50,20.900000,-92.800003,1009,NOT_NAMED,200001,TD,...,-999.0,-999.0,-999.000000,0.170507,0.000000,0.002127,73.992012,46.243252,41.475574,355
2,ATL_200001_TD_2000060806,ATL,2000060806,2451703.75,20.700001,-93.099998,1010,NOT_NAMED,200001,TD,...,-999.0,-999.0,5.240930,0.222971,0.003155,0.009571,50.523193,31.755732,23.392859,125
3,ATL_200001_TD_2000060812,ATL,2000060812,2451704.00,20.799999,-93.500000,1010,NOT_NAMED,200001,TD,...,-999.0,-999.0,11.947744,0.406239,0.419558,0.056009,60.004158,49.730690,45.643269,25
4,ATL_200002_TD_2000062300,ATL,2000062300,2451718.50,9.500000,-19.799999,1010,NOT_NAMED,200002,TD,...,-999.0,-999.0,16.662044,0.538462,0.514196,0.118398,58.072662,42.746475,35.039673,85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40973,SPA_201927_TD_2019051506,SPA,2019051506,2458618.75,-12.600000,142.399994,1008,ANN,201927,TD,...,-999.0,-999.0,7.343043,0.202056,0.031546,0.003899,45.503929,52.581776,55.176037,85
40974,SPA_201927_TD_2019051512,SPA,2019051512,2458619.00,-12.200000,141.000000,1009,ANN,201927,TD,...,-999.0,-999.0,23.535000,0.271889,0.050473,0.008862,50.257538,51.594139,51.143478,75
40975,SPA_201927_TD_2019051518,SPA,2019051518,2458619.25,-11.800000,139.600006,1009,ANN,201927,TD,...,-999.0,-999.0,8.669576,0.269408,0.182965,0.021624,66.502739,54.826412,49.010811,25
40976,SPA_201927_TD_2019051600,SPA,2019051600,2458619.50,-11.200000,138.500000,1008,ANN,201927,TD,...,-999.0,-999.0,-999.000000,0.213400,0.000000,0.013470,68.643570,52.206963,44.478611,185


In [10]:
original_data = original_data.merge(f, on = "GIS_ID")
original_data

,GIS_ID,BASIN,DATE,JULDAY,LATITUDE,LONGITUDE,MSLP,STORM_NAME,STORM_ID,CAT,...,HRA100,HRA300,RAIN_AXIS_100,RAIN_AXIS_300,RAIN_AXIS_100_300,RMR,LOP06,LOP12,LOP18,LOP24
0,ATL_200001_TD_2000060718,ATL,2000060718,2451703.25,21.000000,-93.000000,1008,NOT_NAMED,200001,TD,...,0.151420,0.055654,47.571724,34.547775,28.093451,85,Ocean,Ocean,Ocean,-999
1,ATL_200001_TD_2000060800,ATL,2000060800,2451703.50,20.900000,-92.800003,1009,NOT_NAMED,200001,TD,...,0.000000,0.002127,73.992012,46.243252,41.475574,355,Ocean,Ocean,-999,-999
2,ATL_200001_TD_2000060806,ATL,2000060806,2451703.75,20.700001,-93.099998,1010,NOT_NAMED,200001,TD,...,0.003155,0.009571,50.523193,31.755732,23.392859,125,Ocean,-999,-999,-999
3,ATL_200001_TD_2000060812,ATL,2000060812,2451704.00,20.799999,-93.500000,1010,NOT_NAMED,200001,TD,...,0.419558,0.056009,60.004158,49.730690,45.643269,25,-999,-999,-999,-999
4,ATL_200002_TD_2000062300,ATL,2000062300,2451718.50,9.500000,-19.799999,1010,NOT_NAMED,200002,TD,...,0.514196,0.118398,58.072662,42.746475,35.039673,85,Ocean,Ocean,Ocean,Ocean
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40973,SPA_201927_TD_2019051506,SPA,2019051506,2458618.75,-12.600000,142.399994,1008,ANN,201927,TD,...,0.031546,0.003899,45.503929,52.581776,55.176037,85,Ocean,Ocean,Ocean,-999
40974,SPA_201927_TD_2019051512,SPA,2019051512,2458619.00,-12.200000,141.000000,1009,ANN,201927,TD,...,0.050473,0.008862,50.257538,51.594139,51.143478,75,Ocean,Ocean,-999,-999
40975,SPA_201927_TD_2019051518,SPA,2019051518,2458619.25,-11.800000,139.600006,1009,ANN,201927,TD,...,0.182965,0.021624,66.502739,54.826412,49.010811,25,Ocean,-999,-999,-999
40976,SPA_201927_TD_2019051600,SPA,2019051600,2458619.50,-11.200000,138.500000,1008,ANN,201927,TD,...,0.000000,0.013470,68.643570,52.206963,44.478611,185,-999,-999,-999,-999


In [11]:
original_data.to_csv("HPCNN/IMERG/Land_Ocean_Futures.csv")